In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
from numpy import asarray
from skimage.transform import rescale, resize

# emotion_to_index = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':6, 'sad':4, 'surprise':5}
emotion_to_index = {'Angry':0, 'Disgust':1, 'Fear':2, 'Happy':3, 'Neutral':6, 'Sad':4, 'Surprise':5}
Resize_pixelsize = 96

src = "/content/drive/MyDrive/DATASET/online split"
Y_test = []
X_test = []
X_train = []
Y_train = []
X_val = []
Y_val = []

emotions  =  os.listdir(os.path.join(src,"train"))
for emotion in emotions:
  print(emotion)
  dir = os.listdir(os.path.join(src,"train",emotion))
  for image in dir:
    img = cv2.imread("/content/drive/MyDrive/DATASET/online split/train"+"/"+emotion+"/"+image)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = asarray(gray_image)  
    if(data.size!=65536):
      continue
    image_resized = resize(data, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
    data = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)
    # data = data/255.0
    X_train.append(data)
    Y_train.append(emotion_to_index[emotion])

X_train= np.array(X_train)
Y_train = np.array(Y_train)



# src = "/content/drive/MyDrive/DATASET/Split"
emotions  =  os.listdir(os.path.join(src,"test"))
for emotion in emotions:
  print(emotion)
  dir = os.listdir(os.path.join(src,"test",emotion))
  for image in dir:
    img = cv2.imread("/content/drive/MyDrive/DATASET/online split/test"+"/"+emotion+"/"+image)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = asarray(gray_image)

    if(data.size!=65536):
      continue
    image_resized = resize(data, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
    data = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)
    # data = data/255.0
    X_test.append(data)
    Y_test.append(emotion_to_index[emotion])

X_test= np.array(X_test)
Y_test = np.array(Y_test)


Angry
Disgust
Fear
Happy
Neutral
Sad
Surprise
Angry
Disgust
Fear
Happy
Neutral
Sad
Surprise


In [ ]:
print(X_train.shape)

(1974, 96, 96, 1)


In [ ]:
from keras.utils.np_utils import to_categorical
Y_train = to_categorical(Y_train, num_classes=7)
Y_test = to_categorical(Y_test, num_classes=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Flatten, Dropout, Dense
from tensorflow.keras import optimizers


def get_model(img_x, img_y):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(img_x, img_y, 1)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Conv2D(256, kernel_size=(5, 5),  padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Flatten())
    model.add(Dense(300, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(7, activation='softmax'))

    adam = optimizers.SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  # loss='mse',
                  metrics=['accuracy'])

    return model

In [ ]:
from keras.models import load_model

def train():
    print('loading data...')

    img_x, img_y = 96, 96
    # x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
    # x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)

    print(X_train.shape)
    print(Y_train.shape)

    print('Loading model...')
    model = get_model(96, 96)

    print('Begin training...')
    model.fit(X_train, Y_train, batch_size=64, epochs=50)

    model.save('model.h5')

    score = model.evaluate(X_test, Y_test)
    print('acc', score[1])

In [ ]:
train()

loading data...
(1974, 96, 96, 1)
(1974, 7)
Loading model...
Begin training...
Epoch 1/50
31/31 [==============================] - 14s 331ms/step - loss: 1.9160 - accuracy: 0.2087
Epoch 2/50
31/31 [==============================] - 9s 282ms/step - loss: 1.8908 - accuracy: 0.2194
Epoch 3/50
31/31 [==============================] - 9s 282ms/step - loss: 1.7799 - accuracy: 0.2893
Epoch 4/50
31/31 [==============================] - 9s 282ms/step - loss: 1.6614 - accuracy: 0.3627
Epoch 5/50
31/31 [==============================] - 9s 281ms/step - loss: 1.5230 - accuracy: 0.4296
Epoch 6/50
31/31 [==============================] - 9s 281ms/step - loss: 1.4037 - accuracy: 0.4878
Epoch 7/50
31/31 [==============================] - 9s 282ms/step - loss: 1.2695 - accuracy: 0.5253
Epoch 8/50
31/31 [==============================] - 9s 281ms/step - loss: 1.2085 - accuracy: 0.5633
Epoch 9/50
31/31 [==============================] - 9s 282ms/step - loss: 1.0907 - accuracy: 0.6044
Epoch 10/50
31/31 [=